## This notebook will guide you through logging your experiments

In [1]:
# Import libraries 

# The Training Libs
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env

from stable_baselines3.common.vec_env import VecVideoRecorder

# Import wandb stuff
import wandb
from wandb.integration.sb3 import WandbCallback

## Weights and biases (wandb)

[Weights and Biases](https://wandb.ai/) is a free and open source logging platform that allows you to track multiple results when you are training your model. It collects various metrics, and organizes them in an simple web dashboard. 

In this notebook, we will see how simple it is to start tracking your experiments with weights and biases

It can log all the training metrics that is output during the training phase, along with a video of our agent at that point in our training. Additionally, it also logs system status like CPU/GPU usage and temperature.

They have good documentation, which can be found here - https://docs.wandb.ai/

## Setup for wandb

1. Create an account on wandb - https://wandb.ai/

2. Install wandb: 

    Wandb is distributed as a python package. To install it, run `pip install wandb`.
    
3. Login into wandb:

    To login into wandb, run `wandb login`. Paste the api key. The key can be found here - https://wandb.ai/authorize

In [2]:
# Initilaize wandb
# https://docs.wandb.ai/guides/integrations/other/stable-baselines-3

config = {
    "policy_type": "MlpPolicy",
    "total_timesteps": 100000,
    "env_name": "LunarLander-v2",
}

run = wandb.init(
    project="LunarLander-v2", # It creates a project on wandb if it doesnt exist. The logging happens there
    config=config,
    sync_tensorboard=True,  # auto-upload tensorboard metrics
    monitor_gym=True,  # auto-upload the videos of agents playing the game
)

# After running this cell, go to https://wandb.ai/home to see your new project created

wandb: Currently logged in as: supersecurehuman. Use `wandb login --relogin` to force relogin


## Now, we will create a environment and a model to log the training

This part is covered in the main notebook


In [3]:
env = make_vec_env('LunarLander-v2', n_envs=16)
# Use the folling line with caution. The video recorder will try to render the agent on the screen, so that ffmpeg can caputre it. Here, we have 16 envs set. Trying to render 16 envs on screen will
# be pretty resource intensive. 
# env = VecVideoRecorder(env, f"videos/{run.id}", record_video_trigger=lambda x: x % 2000 == 0, video_length=200) # Set the video recorder, to record our agent during training

model = PPO(
    policy = config["policy_type"],
    env = env,
    tensorboard_log="logs",
    verbose=1)

Using cuda device


In [ ]:
# Now we do the magical stuff of logging to wandb. All you have to do is add the wandb callback to the model's callback like this

model.learn(total_timesteps=config["total_timesteps"], callback=[WandbCallback()])

In [5]:
# Finish run
run.finish()

# This cell output will also give a global summary, along with giving you the link to view your run.

global_step,▁▃▆█
rollout/ep_len_mean,▁▃▄█
rollout/ep_rew_mean,▁▄▆█
time/fps,█▁▁▁
train/approx_kl,▁▇█
train/clip_fraction,▁▅█
train/clip_range,▁▁▁
train/entropy_loss,▁▄█
train/explained_variance,▁▇█
train/learning_rate,▁▁▁
train/loss,█▅▁


### Note

You need to enable tensorboard logging to view your training metrics in wandb dashboard.

## Congarts!

Now you have now started to use wandb in your project. Do checkout the docs to know what are the other amazing stuff it is capable off!